In [ ]:
import os
import gc
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121, InceptionV3, EfficientNetB0, Xception, MobileNetV2
from tensorflow.keras import models, layers, optimizers, callbacks
from tensorflow.keras import backend as K
import pickle
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb




In [2]:
# Criando função para calcular as métricas
def get_metrics(y_true, y_pred):
    vn, fp, fn, vp = confusion_matrix(y_true, y_pred).ravel()
    accuracy = (vp + vn) / (vp + fp + fn + vn)
    recall = vp / (vp + fn)
    specificity = vn / (vn + fp)
    precision = vp / (vp + fp)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    auc_roc = roc_auc_score(y_true, y_pred)

    return {
        'accuracy': accuracy,
        'specificity': specificity,
        'recall': recall,
        'precision': precision,
        'f1-score': f1,
        'kappa': kappa,
        'auc-roc': auc_roc
    }

In [3]:
# Criando função para seleção de esquema de cor 
def convert_color_scale(image, scale):
    if scale == 'hsv':
        return cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    elif scale == 'rgb':
        return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    elif scale == 'grayscale':
        # Converter para escala de cinza e replicar para 3 canais
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        return cv2.merge([gray, gray, gray])
    elif scale == 'lab':
        return cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    elif scale == 'luv':
        return cv2.cvtColor(image, cv2.COLOR_BGR2LUV)
    elif scale == 'xyz':
        return cv2.cvtColor(image, cv2.COLOR_BGR2XYZ)
    elif scale == 'YcrCb':
        return cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    
    else:
        raise ValueError("Escala de cor não suportada.")

In [4]:
# Carregamento e pré-processamento de imagens com escolha de escala de cor
def load_images(folder, color_scale, img_extensions):
    images = []
    for filename in os.listdir(folder):
        if any(filename.lower().endswith(ext) for ext in img_extensions):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)
            
            if img is not None:
                img = cv2.resize(img, (224, 224))  # Ajuste o tamanho conforme necessário

                # Converta para a escala de cor desejada
                img = convert_color_scale(img, color_scale)

                # Se a imagem estiver em escala de cinza, expanda as dimensões
                if color_scale == 'grayscale':
                    img = np.expand_dims(img, axis=-1)  # Adiciona uma dimensão de canal

                images.append(img)
    return np.array(images)

In [5]:
# Defina as pastas de dados
data_dir = r"C:\Users\andre\Pictures\OvarianCancer"
normal_dir = os.path.join(data_dir, 'Non_Cancerous')
cancer_dir = os.path.join(data_dir, 'Serous')
img_extensions = ['.jpg', '.jpeg', '.png']


In [6]:
# Criar um DataFrame para armazenar os resultados
columns = ['Modelo', 'Acuracia', 'Sensibilidade', 'Especificidade', 'F-Score', 'AUC-ROC']
df_metrics = pd.DataFrame(columns=columns)


In [7]:
# Carregar o modelo InceptionV3 pré-treinado
base_model = VGG16(weights='imagenet', include_top=False)     # <----------------------------------------------------------

# Extrair características usando o modelo VGG16
def extract_features(model, X):
    features = model.predict(X)
    return features.reshape((features.shape[0], -1))

In [8]:
# Carregamento de imagens e conversão para XYZ
normal_images = load_images(normal_dir, 'grayscale',img_extensions)        # <----------------------------------------------------------
cancer_images = load_images(cancer_dir, 'grayscale',img_extensions)        # hsv<----------------------------------------------------------


# Rótulos para imagens (0 para normal, 1 para câncer)
normal_labels = np.zeros(normal_images.shape[0])
cancer_labels = np.ones(cancer_images.shape[0])

# Concatenar imagens e rótulos
all_images = np.concatenate([normal_images, cancer_images], axis=0)
all_images = extract_features(base_model, all_images)
all_labels = np.concatenate([normal_labels, cancer_labels], axis=0)

# Dividir o conjunto de dados em treino e teste (80/20)
X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)

X_train = X_train / 255.0
X_test = X_test / 255.0


In [ ]:
# Classificação com 5 classificadores
# Para o SVM
svm_model = SVC(random_state=42, kernel='rbf', C=1, gamma='scale')
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
metrics_svm = get_metrics(y_test, y_pred_svm)
print("Métricas para SVM:")
print(metrics_svm)

# Para o Random Forest
rf_model = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
metrics_rf = get_metrics(y_test, y_pred_rf)
print("Métricas para Random Forest:")
print(metrics_rf)

# Para o KNN
knn_model = KNeighborsClassifier(n_neighbors=4)
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)
metrics_knn = get_metrics(y_test, y_pred_knn)
print("Métricas para K-Nearest Neighbors (KNN):")
print(metrics_knn)

# Para o AdaBoost
adaboost_model = AdaBoostClassifier(random_state=42, n_estimators=50, learning_rate=1.0)#  random_state=42, n_estimators=100, learning_rate=0.1
adaboost_model.fit(X_train, y_train)
y_pred_adaboost = adaboost_model.predict(X_test)
metrics_adaboost = get_metrics(y_test, y_pred_adaboost)
print("Métricas para AdaBoost (Gradient Boosting):")
print(metrics_adaboost)

# Para o XGB
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42, max_depth=9, 
                              colsample_bytree=0.4033, min_child_weight=6, gamma=0.429, 
                              eta=0.5995, n_estimators=1000, use_label_encoder=False, 
                              eval_metric='merror')
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
metrics_xgb = get_metrics(y_test, y_pred_xgb)
print("Métricas para XGBoost:")
print(metrics_xgb)

# Criar DataFrames e salvar os resultados em um xlsx 
df_metrics_svm = pd.DataFrame([metrics_svm.values()], columns=metrics_svm.keys())
df_metrics_svm.insert(0, 'Classificador', 'SVM')

df_metrics_rf = pd.DataFrame([metrics_rf.values()], columns=metrics_rf.keys())
df_metrics_rf.insert(0, 'Classificador', 'RF')

df_metrics_knn = pd.DataFrame([metrics_knn.values()], columns=metrics_knn.keys())
df_metrics_knn.insert(0, 'Classificador', 'KNN')

df_metrics_adaboost = pd.DataFrame([metrics_adaboost.values()], columns=metrics_adaboost.keys())
df_metrics_adaboost.insert(0, 'Classificador', 'AdaBoost')

df_metrics_xgb = pd.DataFrame([metrics_xgb.values()], columns=metrics_xgb.keys())
df_metrics_xgb.insert(0, 'Classificador', 'XGBoost')

# Concatenar todos os DataFrames
df_all_metrics = pd.concat([df_metrics_svm, df_metrics_rf, df_metrics_knn, df_metrics_adaboost, df_metrics_xgb])

# Salvar o DataFrame em um arquivo Excel
'df_all_metrics.to_excel('testando.xlsx', index=False)  # <----------------------------------------------------------

df_all_metrics


Métricas para SVM:
{'accuracy': 0.95, 'specificity': 0.9523809523809523, 'recall': 0.9473684210526315, 'precision': 0.9473684210526315, 'f1-score': 0.9473684210526315, 'kappa': 0.899749373433584, 'auc-roc': 0.9498746867167918}
Métricas para Random Forest:
{'accuracy': 0.925, 'specificity': 0.9523809523809523, 'recall': 0.8947368421052632, 'precision': 0.9444444444444444, 'f1-score': 0.918918918918919, 'kappa': 0.8492462311557789, 'auc-roc': 0.9235588972431078}
Métricas para K-Nearest Neighbors (KNN):
{'accuracy': 0.75, 'specificity': 0.6666666666666666, 'recall': 0.8421052631578947, 'precision': 0.6956521739130435, 'f1-score': 0.761904761904762, 'kappa': 0.5037220843672456, 'auc-roc': 0.7543859649122807}
Métricas para AdaBoost (Gradient Boosting):
{'accuracy': 0.95, 'specificity': 1.0, 'recall': 0.8947368421052632, 'precision': 1.0, 'f1-score': 0.9444444444444444, 'kappa': 0.8992443324937027, 'auc-roc': 0.9473684210526316}
Métricas para XGBoost:
{'accuracy': 0.925, 'specificity': 0.952

,Classificador,accuracy,specificity,recall,precision,f1-score,kappa,auc-roc
0,SVM,0.950,0.952381,0.947368,0.947368,0.947368,0.899749,0.949875
0,RF,0.925,0.952381,0.894737,0.944444,0.918919,0.849246,0.923559
0,KNN,0.750,0.666667,0.842105,0.695652,0.761905,0.503722,0.754386
0,AdaBoost,0.950,1.000000,0.894737,1.000000,0.944444,0.899244,0.947368
0,XGBoost,0.925,0.952381,0.894737,0.944444,0.918919,0.849246,0.923559


In [9]:
# Defina o número de splits para a validação cruzada
from sklearn.model_selection import KFold
num_splits = 5
kf = KFold(n_splits=num_splits, shuffle=True, random_state=42)

# Crie uma lista para armazenar os resultados de cada fold
metrics_list = []

X = all_images
y = all_labels

for fold, (train_index, test_index) in enumerate(kf.split(X), 1):
    # Divida os dados em treino e teste
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Crie e treine o modelo
    svm_model =SVC(random_state=42, kernel='rbf', C=1, gamma='scale')
    svm_model.fit(X_train, y_train)
    
    # Faça a predição
    y_pred = svm_model.predict(X_test)
    
    # Calcule as métricas
    metrics = get_metrics(y_test, y_pred)
    metrics_list.append(metrics)

# Crie um DataFrame para armazenar os resultados de cada fold
metrics_df = pd.DataFrame(metrics_list)

# Calcule a média e o desvio padrão
mean_metrics = metrics_df.mean()
std_metrics = metrics_df.std()

# Adicione as linhas de média e desvio padrão ao DataFrame
metrics_df.loc['Média'] = mean_metrics
metrics_df.loc['Desvio Padrão'] = std_metrics

print("Métricas para cada fold:")
metrics_df



Métricas para cada fold:


,accuracy,specificity,recall,precision,f1-score,kappa,auc-roc
0,0.925000,1.000000,0.842105,1.000000,0.914286,0.848485,0.921053
1,0.875000,0.888889,0.863636,0.904762,0.883721,0.748744,0.876263
2,0.875000,0.750000,1.000000,0.800000,0.888889,0.750000,0.875000
3,0.925000,0.913043,0.941176,0.888889,0.914286,0.847716,0.927110
4,0.897436,0.823529,0.954545,0.875000,0.913043,0.788618,0.889037
Média,0.899487,0.875092,0.920293,0.893730,0.902845,0.796712,0.897693
Desvio Padrão,0.025026,0.094259,0.065738,0.071719,0.015218,0.049574,0.024800


In [ ]:

# menor=1
while maior_acc_test <0.92:
  X_train, X_test, Y_train, Y_test = train_test_split(all_images, all_labels,test_size=0.1)



  k_range = list(range(1, 30))

  maior_k = 0.0
  maior_acc_test = 0.0

  # print("k\t\tAcc Test\t\tAcc Train")

  for k in k_range:
      knn = KNeighborsClassifier(n_neighbors=k)
      knn = knn.fit(X_train, Y_train)
      acc_test = knn.score(X_test, Y_test)
      acc_train = knn.score(X_train, Y_train)
      
      if acc_test > maior_acc_test:
          maior_k = k
          maior_acc_test = acc_test

    #   if acc_test <menor:
    #      menor = acc_test
      
      # print(str(k)+"\t\t"+str(acc_test)+"\t\t"+str(acc_train))
if maior_acc_test>=0.8:
#   print("variaçaão",maior_acc_test-menor)
  print("Maior valor de K é", maior_k, "com acurácia em teste igual a", maior_acc_test)


NameError: name 'maior_acc_test' is not defined

In [ ]:

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

def get_gradcam(model, img_array, layer_name):
    grad_model = tf.keras.models.Model(
        inputs=model.input, 
        outputs=[model.get_layer(layer_name).output, model.output]
    )
    
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, 1]  # Supondo que a classe positiva (câncer) seja o índice 1

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_outputs), axis=-1).numpy().squeeze()
    heatmap = np.maximum(heatmap, 0)  # ReLU
    heatmap /= np.max(heatmap)  # Normalizar

    return heatmap

def overlay_heatmap(img, heatmap, alpha=0.4):
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    
    overlayed = cv2.addWeighted(img, 1 - alpha, heatmap, alpha, 0)
    return overlayed

# Exemplo de uso
img_path = r"C:\Users\andre\Pictures\OvarianCancer\Serous\high gradw serous.JPG"  # Substituir pelo caminho da imagem
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_array = cv2.resize(img, (224, 224))  # Ajustar ao tamanho esperado pelo modelo
img_array = np.expand_dims(img_array, axis=0) / 255.0

layer_name = 'block5_conv3'  # Substituir por uma camada convolucional do modelo usado
heatmap = get_gradcam(base_model, img_array, layer_name)
overlayed_img = overlay_heatmap(img, heatmap)

plt.figure(figsize=(10,5))
plt.subplot(1, 2, 1)
plt.imshow(img)
plt.title("Imagem Original")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(overlayed_img)
plt.title("Grad-CAM")
plt.axis("off")

plt.show()


error: OpenCV(4.5.3) D:\bld\libopencv_1632857399304\work\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


NameError: name 'model' is not defined